In [ ]:
#------------------------------------------------------------------------------------------------------------------------
# Import packages
#------------------------------------------------------------------------------------------------------------------------

!pip install tensorflow
!pip install keras

import time
import tensorflow as tf
import scipy.io as sio
import keras
tf.keras.backend.clear_session()
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D, concatenate, Flatten, Reshape, Cropping2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/gdrive')

!nvidia-smi
tf.test.gpu_device_name()





#------------------------------------------------------------------------------------------------------------------------
# To define
#------------------------------------------------------------------------------------------------------------------------

run_no = "7"

training_dataset_path_noisefree = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/noisefree.mat"
training_dataset_path_noisy = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/noisy.mat"
checkpoint_path = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + ""


# set the number of epochs
Nepochs = 500





#------------------------------------------------------------------------------------------------------------------------
# Load the training data
#------------------------------------------------------------------------------------------------------------------------

X = sio.loadmat(training_dataset_path_noisefree)
X = X['I_Abel_quadrant']
X = np.swapaxes(X, 0, -1)
X = np.swapaxes(X, 1, 2)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

X_noisy = sio.loadmat(training_dataset_path_noisy)
X_noisy = X_noisy['I_Abel_withnoise_quadrant']

X_noisy = np.swapaxes(X_noisy, 0, -1)
X_noisy = np.swapaxes(X_noisy, 1, 2)
X_noisy = np.reshape(X_noisy, (X.shape[0], X.shape[1], X.shape[2], 1))

# X = X[:500]
# X_noisy = X_noisy[:500]
# X_training = X[50:]
# X_training_noisy = X_noisy[50:]
# X_validation = X[:50]
# X_validation_noisy = X_noisy[:50]

X = X[:2000]
X_noisy = X_noisy[:2000]
X_training = X[200:]
X_training_noisy = X_noisy[200:]
X_validation = X[:200]
X_validation_noisy = X_noisy[:200]


# model
def create_model():
    UB = True
    w = 5
    inputs = Input((90,90,1))
    conv1 = Conv2D(64, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(inputs)
    pool1 = MaxPool2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(128, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool1)
    pool2 = MaxPool2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(256, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool2)
    pool3 = MaxPool2D((2,2), padding='same')(conv3)

    conv4 = Conv2D(512, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool3)
    pool4 = MaxPool2D((2,2), padding='same')(conv4)

    conv5 = Conv2D(1024, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool4)

    deconv4 = Conv2DTranspose(512, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(conv5)
    cropped_deconv4 = Cropping2D(cropping=((0, 0), (0, 0)))(deconv4)
    merge4 = concatenate([conv4,cropped_deconv4], axis = 3)

    deconv3 = Conv2DTranspose(256, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge4)
    cropped_deconv3 = Cropping2D(cropping=((1, 0), (1, 0)))(deconv3)
    merge3 = concatenate([conv3,cropped_deconv3], axis = 3)

    deconv2 = Conv2DTranspose(128, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge3)
    cropped_deconv2 = Cropping2D(cropping=((1, 0), (1, 0)))(deconv2)
    merge2 = concatenate([conv2,cropped_deconv2], axis = 3)

    deconv1 = Conv2DTranspose(64, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge2)
    cropped_deconv1 = Cropping2D(cropping=((0, 0), (0, 0)))(deconv1)
    merge1 = concatenate([conv1,cropped_deconv1], axis = 3)

    conv_last = Conv2D(1, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(merge1)

    model = Model(inputs,conv_last)

    model.compile(optimizer = 'Adam', loss = "mean_squared_error", metrics = ["accuracy"])

    return model

model = create_model()

model.summary()


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq = 250,
                                                 verbose = 1)

start_time = time.time()
history = model.fit(X_training_noisy, X_training, validation_data = (X_validation_noisy, X_validation), epochs = Nepochs, batch_size = 32, callbacks=[cp_callback])

training_time = time.time()-start_time

# test and save on validation data
X_val_noisy = X_validation_noisy
X_val = X_validation
model.load_weights(checkpoint_path)
start_time = time.time()
predict = model.predict(X_val_noisy)
testing_time = time.time()-start_time
dictionary = {}
dictionary['results'] = {'input':X_val_noisy, 'predictions':predict, 'validation':X_val}
dictionary['time'] = {'training_time': training_time, 'testing_time': testing_time}

sio.savemat("/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/results.mat", dictionary)
sio.savemat("/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/history.mat", history.history)

Mounted at /content/gdrive
Tue Apr 30 08:37:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+----------------------------------------

In [ ]:
#install keras and tensorflow
!pip install tensorflow
# !pip install tensorflow==2.5.0
!pip install keras

# !pip install keras==2.4.3

# !pip install tensorflow==2.1.0
# !pip install keras==2.2.4

import os
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Assign which GPU to run on, "0" or "1"
import tensorflow as tf
import scipy.io as sio
# from tensorflow import keras
import keras
print(tf.__version__)
print(keras.__version__)
tf.keras.backend.clear_session()
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D, concatenate, Flatten, Reshape, Cropping2D
from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers.legacy.adam import Adam

from google.colab import drive
drive.mount('/content/gdrive')
# !unzip "/content/training_data.zip" -d "/content"

!nvidia-smi
tf.test.gpu_device_name()





#------------------------------------------------------------------------------------------------------------------------
# To define
#------------------------------------------------------------------------------------------------------------------------

run_no = "7"

training_dataset_path_noisefree = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/noisefree.mat"
training_dataset_path_noisy = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + "/noisy.mat"
checkpoint_path = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no + ""


# set the number of epochs
Nepochs = 500





#------------------------------------------------------------------------------------------------------------------------
# Load the training data
#------------------------------------------------------------------------------------------------------------------------

X = sio.loadmat(training_dataset_path_noisefree)
X = X['I_Abel_quadrant']
X = np.swapaxes(X, 0, -1)
X = np.swapaxes(X, 1, 2)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

X_noisy = sio.loadmat(training_dataset_path_noisy)
X_noisy = X_noisy['I_Abel_withnoise_quadrant']

X_noisy = np.swapaxes(X_noisy, 0, -1)
X_noisy = np.swapaxes(X_noisy, 1, 2)
X_noisy = np.reshape(X_noisy, (X.shape[0], X.shape[1], X.shape[2], 1))

X = X[:2000]
X_noisy = X_noisy[:2000]
X_training = X[200:]
X_training_noisy = X_noisy[200:]
X_validation = X[:200]
X_validation_noisy = X_noisy[:200]


# model
def create_model():
    UB = True
    w = 5
    inputs = Input((90,90,1))
    conv1 = Conv2D(64, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(inputs)
    pool1 = MaxPool2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(128, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool1)
    pool2 = MaxPool2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(256, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool2)
    pool3 = MaxPool2D((2,2), padding='same')(conv3)

    conv4 = Conv2D(512, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool3)
    pool4 = MaxPool2D((2,2), padding='same')(conv4)

    conv5 = Conv2D(1024, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool4)

    deconv4 = Conv2DTranspose(512, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(conv5)
    cropped_deconv4 = Cropping2D(cropping=((0, 0), (0, 0)))(deconv4)
    merge4 = concatenate([conv4,cropped_deconv4], axis = 3)

    deconv3 = Conv2DTranspose(256, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge4)
    cropped_deconv3 = Cropping2D(cropping=((1, 0), (1, 0)))(deconv3)
    merge3 = concatenate([conv3,cropped_deconv3], axis = 3)

    deconv2 = Conv2DTranspose(128, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge3)
    cropped_deconv2 = Cropping2D(cropping=((1, 0), (1, 0)))(deconv2)
    merge2 = concatenate([conv2,cropped_deconv2], axis = 3)

    deconv1 = Conv2DTranspose(64, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge2)
    cropped_deconv1 = Cropping2D(cropping=((0, 0), (0, 0)))(deconv1)
    merge1 = concatenate([conv1,cropped_deconv1], axis = 3)

    conv_last = Conv2D(1, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(merge1)

    model = Model(inputs,conv_last)

    model.compile(optimizer = 'Adam', loss = "mean_squared_error", metrics = ["accuracy"])

    return model

model = create_model()

model.summary()


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq = 250,
                                                 verbose = 1)

start_time = time.time()




process_dataset_path_noisy = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no +"/noisy_process.mat"
# process_dataset_path_noisy = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no +"/noisy_process_real.mat"
X_val_noisy = sio.loadmat(process_dataset_path_noisy)
X_val_noisy = X_val_noisy['process_quadrant']
X_val_noisy = np.swapaxes(X_val_noisy, 0, -1)
X_val_noisy = np.swapaxes(X_val_noisy, 1, 2)
X_val_noisy = np.reshape(X_val_noisy, (X_val_noisy.shape[0], X_val_noisy.shape[1], X_val_noisy.shape[2], 1))

process_dataset_path_noisefree = "/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no +"/noisefree.mat"
X_val = sio.loadmat(process_dataset_path_noisefree)
X_val = X_val['I_Abel_quadrant']
X_val = np.swapaxes(X_val, 0, -1)
X_val = np.swapaxes(X_val, 1, 2)
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))

model.load_weights(checkpoint_path)
start_time = time.time()
predict = model.predict(X_val_noisy)
testing_time = time.time()-start_time
dictionary = {}
dictionary['results'] = {'input':X_val_noisy, 'predictions':predict, 'validation':X_val}
# dictionary['time'] = {'training_time': training_time, 'testing_time': testing_time}

sio.savemat("/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no +"/results_process.mat", dictionary)
# sio.savemat("/content/gdrive/MyDrive/PhD/denoise_oxford/run_" + run_no +"/history.mat", history.history)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 